In [1]:
#!/usr/bin/env python3
"""
TSP FunSearch Runner Script

This script sets up and runs a FunSearch experiment for the Traveling Salesman Problem.
It uses the provided LLM and Sandbox implementations to evolve sophisticated heuristic
algorithms for the TSP problem.
"""

import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Add project root to path
sys.path.append('.')

# Import custom implementations
from implementation.sampler import LLM
from implementation.evaluator import Sandbox

# Import FunSearch components
from implementation import config as config_lib
from implementation import funsearch

AttributeError: module 'implementation.config' has no attribute 'ClassConfig'

In [ ]:
# TSP specification with the function to evolve
TSP_SPECIFICATION = r'''
import numpy as np
import random

def heuristic_tsp_solver(distances, priority_func):
    """Solving the TSP Problem Using Enhanced Heuristics"""
    num_cities = len(distances)
    best_tour = None
    best_distance = float('inf')

    # Try several different starting points to find a better solution
    start_cities = [0, random.randint(0, num_cities-1), num_cities//2]
    for start_city in start_cities:
        # Build the path from the starting city
        visited = [False] * num_cities
        current_city = start_city
        visited[current_city] = True
        tour = [current_city]
        total_distance = 0

        # Building a complete pathway
        while len(tour) < num_cities:
            # Determining the next city using the priority function
            priorities = priority_func(current_city, distances, visited)
            masked_priorities = np.where(visited, np.inf, priorities)
            next_city = np.argmin(masked_priorities)

            # Update Path
            visited[next_city] = True
            tour.append(next_city)
            total_distance += distances[current_city][next_city]
            current_city = next_city

        # closed loop
        total_distance += distances[current_city][tour[0]]

        # Save the best path
        if total_distance < best_distance:
            best_distance = total_distance
            best_tour = tour.copy()

    # Simple 2-opt optimisation (if there is enough time)
    best_tour, best_distance = two_opt_improvement(best_tour, distances, max_iterations=50)

    return best_tour, best_distance

def two_opt_improvement(tour, distances, max_iterations=100):
    """Optimising a given travel path using the 2-opt algorithm"""
    n = len(tour)
    best_tour = tour.copy()
    best_distance = calculate_tour_distance(best_tour, distances)
    improved = True
    iterations = 0

    while improved and iterations < max_iterations:
        improved = False
        iterations += 1

        for i in range(1, n - 1):
            for j in range(i + 1, n):
                # Avoid invalid 2-opt operations (when j is the last city and i is the first city)
                if i == 0 and j == n - 1:
                    continue

                # Creating a new path: reversing the part from i to j
                new_tour = best_tour.copy()
                new_tour[i:j+1] = reversed(best_tour[i:j+1])

                # Calculate the length of the new path
                new_distance = calculate_tour_distance(new_tour, distances)

                # If the new path is better, accept it
                if new_distance < best_distance:
                    best_distance = new_distance
                    best_tour = new_tour
                    improved = True
                    break

            if improved:
                break

    return best_tour, best_distance

def calculate_tour_distance(tour, distances):
    """Calculate the total distance of a given travelling path"""
    total = 0
    for i in range(len(tour) - 1):
        total += distances[tour[i]][tour[i+1]]
    # closed loop
    total += distances[tour[-1]][tour[0]]
    return total

@funsearch.run
def evaluate(distance_matrix):
    """Evaluate the TSP solution on a given distance matrix"""
    tour, total_distance = heuristic_tsp_solver(distance_matrix, priority)
    return -total_distance  # The negative sign is because FunSearch maximises the score

@funsearch.evolve
def priority(current_city, distances, visited):
    """
    Calculates the priority value from the current city to each city.
    Lower values indicate higher priority (will be selected earlier).
    """
    num_cities = len(distances)
    priorities = np.full(num_cities, np.inf)

    # Calculating travel progress
    visited_count = sum(visited)
    progress = visited_count / num_cities

    for city in range(num_cities):
        if not visited[city]:
            # Basic distance factor
            distance = distances[current_city][city]

            # Calculate the average distance from this city to all unvisited cities (connectivity)
            connectivity = 0
            unvisited_count = 0
            for next_city in range(num_cities):
                if not visited[next_city] and next_city != city:
                    connectivity += distances[city][next_city]
                    unvisited_count += 1

            avg_connectivity = connectivity / max(1, unvisited_count)

            # Consider the distance back to the starting point (more important later in the journey)
            completion_factor = 0
            if progress > 0.6:  # When more than 60 per cent of cities are visited
                start_city = 0  # Assume 0 is the starting city
                completion_factor = distances[city][start_city]

            # Adjustment of weights according to travel progress
            w1 = 1.0  # distance weighting
            w2 = 0.8 * (1 - progress)  # Connectivity weighting (more important early in the trip)
            w3 = 2.0 * progress  # Completion of path weights (more important later in the trip)

            # Calculate priority (lower values indicate higher priority)
            priorities[city] = w1 * distance + w2 * avg_connectivity + w3 * completion_factor

    return priorities
'''

In [ ]:
def load_tsp_data(filepath):
    """Load TSP data from a file and convert to distance matrix."""
    # Parse node coordinates
    node_coords = {}
    with open(filepath, "r") as f:
        lines = f.readlines()
    
    found_node_section = False
    for line in lines:
        if found_node_section:
            if line.strip() == "EOF":
                break
            parts = line.strip().split()
            node_id = float(parts[0])
            x = float(parts[1])
            y = float(parts[2])
            node_coords[node_id] = (x, y)
        elif line.startswith("DISPLAY_DATA_SECTION") or line.startswith("NODE_COORD_SECTION"):
            found_node_section = True
    
    # Convert to distance matrix
    return coordinates_to_distance_matrix(node_coords)

def coordinates_to_distance_matrix(coordinates):
    """Convert coordinates to a distance matrix."""
    num_cities = len(coordinates)
    distance_matrix = np.zeros((num_cities, num_cities))
    
    city_ids = sorted(coordinates.keys())
    
    for i in range(num_cities):
        for j in range(i + 1, num_cities):
            coord1 = coordinates[city_ids[i]]
            coord2 = coordinates[city_ids[j]]
            distance = np.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)
            distance_matrix[i][j] = distance_matrix[j][i] = distance
    
    return distance_matrix

def visualize_tour(coordinates, tour, title="Optimized TSP Tour"):
    """Visualize a TSP tour."""
    city_ids = sorted(coordinates.keys())
    coords = [coordinates[city_ids[i]] for i in range(len(city_ids))]
    
    plt.figure(figsize=(10, 8))
    
    # Plot cities
    xs = [coord[0] for coord in coords]
    ys = [coord[1] for coord in coords]
    plt.scatter(xs, ys, c='blue', s=50)
    
    # Plot tour
    tour_coords = [coords[city] for city in tour]
    tour_coords.append(tour_coords[0])  # Close the loop
    
    tour_xs = [coord[0] for coord in tour_coords]
    tour_ys = [coord[1] for coord in tour_coords]
    plt.plot(tour_xs, tour_ys, 'r-', linewidth=1)
    
    # Label cities
    for i, (x, y) in enumerate(coords):
        plt.annotate(str(i), (x, y), textcoords="offset points", xytext=(0,10), ha='center')
    
    plt.title(title)
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.grid(True)
    
    # Save the visualization
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    plt.savefig(f"tsp_tour_{timestamp}.png")
    plt.show()

def main():
    # Create directory for logs if it doesn't exist
    log_dir = "./logs/tsp_funsearch"
    os.makedirs(log_dir, exist_ok=True)
    
    # Load TSP data
    tsp_file = "./data/bayg29.tsp"  # Update with your TSP file path
    distance_matrix = load_tsp_data(tsp_file)
    inputs = {"bayg29.tsp": distance_matrix}
    
    # Configure FunSearch
    config = config_lib.Config(
        samples_per_prompt=2,      # Number of samples per prompt
        num_evaluators=4,          # Number of parallel evaluators
        num_samplers=2,            # Number of parallel samplers
        programs_database=config_lib.ProgramsDatabaseConfig(
            functions_per_prompt=3,  # Number of functions to include in each prompt
            num_islands=6,           # Number of islands for diversity
            reset_period=3600        # Reset period in seconds (1 hour)
        )
    )
    
    # Configure classes
    class_config = config_lib.ClassConfig(
        llm_class=LLM, 
        sandbox_class=Sandbox
    )
    
    # Set experiment parameters
    max_samples = 50               # Max number of samples before stopping
    timeout_seconds = 30           # Timeout for each function evaluation
    
    print(f"Starting FunSearch TSP experiment with {max_samples} samples")
    
    # Run FunSearch
    results = funsearch.main(
        specification=TSP_SPECIFICATION,
        inputs=inputs,
        config=config,
        max_sample_nums=max_samples,
        class_config=class_config,
        log_dir=log_dir,
        evaluate_timeout_seconds=timeout_seconds
    )
    
    if results:
        print("\n" + "="*80)
        print(f"FunSearch experiment completed")
        print(f"Best score: {results['best_score']}")
        print(f"Best island: {results['best_island']}")
        print("="*80 + "\n")
        
        # Extract the best function
        print("Best function found:")
        print(results['best_function'])
        
        # TODO: Optionally visualize the best tour
        # This would require running the best function and extracting the tour
    
    print("FunSearch TSP experiment completed!")

if __name__ == "__main__":
    main()